In [2]:
import findspark

findspark.init()

import pyspark

sc = pyspark.SparkContext(appName="myAppName")

import os

In [3]:
#Shakespeare from Project Guttenberg

if not os.path.exists('shakespeare.txt'):
    ! wget http://www.gutenberg.org/cache/epub/100/pg100.txt -O shakespeare.txt

In [4]:
#Douglass from Project Guttenberg

if not os.path.exists('Douglass1.txt'):
    ! wget http://www.gutenberg.org/files/23/23.txt -O douglass1.txt
        
if not os.path.exists('Douglass2.txt'):
    ! wget http://www.gutenberg.org/files/202/202.txt -O douglass2.txt

--2016-12-21 18:05:09--  http://www.gutenberg.org/files/23/23.txt
Resolving www.gutenberg.org... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248399 (243K) [text/plain]
Saving to: “douglass1.txt”

100%[======================================>] 248,399     1.56M/s   in 0.2s    

2016-12-21 18:05:09 (1.56 MB/s) - “douglass1.txt” saved [248399/248399]

--2016-12-21 18:05:09--  http://www.gutenberg.org/files/202/202.txt
Resolving www.gutenberg.org... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 797605 (779K) [text/plain]
Saving to: “douglass2.txt”

100%[======================================>] 797,605     2.87M/s   in 0.3s    

2016-12-21 18:05:09 (2.87 MB/s) - “douglass2.txt” saved [797605/797605]



In [15]:
shak_doc = sc.textFile('shakespeare.txt')
shak_words = shak_doc.flatMap(lambda x :x.split())
shak_words.cache()
shak_words.count()

904061

In [18]:
shak_words.take(7)

[u'The', u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'Complete']

In [17]:
shakespeare_sevengrams = shak_doc.map(lambda x:x.split()) \
    .flatMap(lambda x: [((x[i],x[i+1],x[i+2],x[i+3],x[i+4],x[i+5],x[i+6])) for i in range(0,len(x)-6)])
shakespeare_sevengrams.take(10)

[(u'The', u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'Complete'),
 (u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'Complete', u'Works'),
 (u'Gutenberg', u'EBook', u'of', u'The', u'Complete', u'Works', u'of'),
 (u'EBook', u'of', u'The', u'Complete', u'Works', u'of', u'William'),
 (u'of', u'The', u'Complete', u'Works', u'of', u'William', u'Shakespeare,'),
 (u'The', u'Complete', u'Works', u'of', u'William', u'Shakespeare,', u'by'),
 (u'This', u'eBook', u'is', u'for', u'the', u'use', u'of'),
 (u'eBook', u'is', u'for', u'the', u'use', u'of', u'anyone'),
 (u'is', u'for', u'the', u'use', u'of', u'anyone', u'anywhere'),
 (u'for', u'the', u'use', u'of', u'anyone', u'anywhere', u'at')]

In [19]:
doug1_doc = sc.textFile('douglass1.txt')
doug1_words = doug1_doc.flatMap(lambda x :x.split())
doug1_words.cache()
doug1_words.count()

43792

In [20]:
doug2_doc = sc.textFile('douglass2.txt')
doug2_words = doug2_doc.flatMap(lambda x :x.split())
doug2_words.cache()
doug2_words.count()

138065

In [30]:
douglass_doc = sc.textFile("douglass1.txt*,douglass2.txt")#easy to load multiple text files in a single Spark RDD
douglass_words = douglass_doc.flatMap(lambda x :x.split())
douglass_words.cache()
douglass_words.count()

181857

In [29]:
doug_words.take(7)

[u'The', u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'Narrative']

In [54]:
douglass_sevengrams = douglass_doc.map(lambda x:x.split()) \
    .flatMap(lambda x: [((x[i],x[i+1],x[i+2],x[i+3],x[i+4],x[i+5],x[i+6])) for i in range(0,len(x)-6)])
douglass_sevengrams.take(10)

[(u'The', u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'Narrative'),
 (u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'Narrative', u'of'),
 (u'Gutenberg', u'EBook', u'of', u'The', u'Narrative', u'of', u'the'),
 (u'EBook', u'of', u'The', u'Narrative', u'of', u'the', u'Life'),
 (u'of', u'The', u'Narrative', u'of', u'the', u'Life', u'of'),
 (u'The', u'Narrative', u'of', u'the', u'Life', u'of', u'Frederick'),
 (u'This', u'eBook', u'is', u'for', u'the', u'use', u'of'),
 (u'eBook', u'is', u'for', u'the', u'use', u'of', u'anyone'),
 (u'is', u'for', u'the', u'use', u'of', u'anyone', u'anywhere'),
 (u'for', u'the', u'use', u'of', u'anyone', u'anywhere', u'at')]

In [55]:
%time douglass_shakespeare_mentions = shakespeare_sevengrams.intersection(douglass_sevengrams)
douglass_shakespeare_mentions.top(10)

CPU times: user 13 ms, sys: 0 ns, total: 13 ms
Wall time: 21.3 ms


[(u'your', u'written', u'explanation.', u'The', u'person', u'or', u'entity'),
 (u'your', u'efforts', u'and', u'donations', u'can', u'help,', u'see'),
 (u'your', u'country', u'in', u'addition', u'to', u'the', u'terms'),
 (u'you!)', u'can', u'copy', u'and', u'distribute', u'it', u'in'),
 (u'you', u'wish', u'to', u'charge', u'a', u'fee', u'or'),
 (u'you', u'share', u'it', u'without', u'charge', u'with', u'others.'),
 (u'you', u'received', u'the', u'work', u'from.', u'If', u'you'),
 (u'you', u'received', u'the', u'work', u'electronically,', u'the', u'person'),
 (u'you', u'receive', u'specific', u'permission.', u'If', u'you', u'do'),
 (u'you', u'paid', u'the', u'fee', u'as', u'set', u'forth')]

In [44]:
#Kafka from Project Guttenberg

if not os.path.exists('kafka.txt'):
    ! wget http://www.gutenberg.org/cache/epub/5200/pg5200.txt -O kafka.txt

kafka_doc = sc.textFile('kafka.txt')
kafka_words = kafka_doc.flatMap(lambda x :x.split())
kafka_words.cache()
kafka_words.count()

--2016-12-21 18:56:38--  http://www.gutenberg.org/cache/epub/5200/pg5200.txt
Resolving www.gutenberg.org... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141420 (138K) [text/plain]
Saving to: “kafka.txt”

100%[======================================>] 141,420     --.-K/s   in 0.06s   

2016-12-21 18:56:38 (2.36 MB/s) - “kafka.txt” saved [141420/141420]



25186

In [45]:
kafka_sevengrams = kafka_doc.map(lambda x:x.split()) \
    .flatMap(lambda x: [((x[i],x[i+1],x[i+2],x[i+3],x[i+4],x[i+5],x[i+6])) for i in range(0,len(x)-6)])
kafka_sevengrams.take(10)

[(u'The', u'Project', u'Gutenberg', u'EBook', u'of', u'Metamorphosis,', u'by'),
 (u'Project',
  u'Gutenberg',
  u'EBook',
  u'of',
  u'Metamorphosis,',
  u'by',
  u'Franz'),
 (u'Gutenberg', u'EBook', u'of', u'Metamorphosis,', u'by', u'Franz', u'Kafka'),
 (u'This', u'eBook', u'is', u'for', u'the', u'use', u'of'),
 (u'eBook', u'is', u'for', u'the', u'use', u'of', u'anyone'),
 (u'is', u'for', u'the', u'use', u'of', u'anyone', u'anywhere'),
 (u'for', u'the', u'use', u'of', u'anyone', u'anywhere', u'at'),
 (u'the', u'use', u'of', u'anyone', u'anywhere', u'at', u'no'),
 (u'use', u'of', u'anyone', u'anywhere', u'at', u'no', u'cost'),
 (u'of', u'anyone', u'anywhere', u'at', u'no', u'cost', u'and')]

In [58]:
%time douglass_shakespeare_mentions_trimmed = douglass_shakespeare_mentions.subtract(kafka_sevengrams, numPartitions=None)

CPU times: user 16 ms, sys: 1 ms, total: 17 ms
Wall time: 22.1 ms


In [50]:
douglass_shakespeare_mentions_trimmed.collect()

[(u'with', u'this', u'eBook', u'or', u'online', u'at', u'www.gutenberg.org'),
 (u'domain', u'print', u'editions', u'means', u'that', u'no', u'one'),
 (u'and', u'research.', u'They', u'may', u'be', u'modified', u'and'),
 (u'printed', u'and', u'given', u'away--you', u'may', u'do', u'practically'),
 (u'distributing',
  u'Project',
  u'Gutenberg-tm',
  u'electronic',
  u'works',
  u'to',
  u'protect'),
 (u'copyright', u'royalties.', u'Special', u'rules,', u'set', u'forth', u'in'),
 (u'reports,', u'performances', u'and', u'research.', u'They', u'may', u'be'),
 (u'damage', u'or', u'cannot', u'be', u'read', u'by', u'your'),
 (u'of', u'this', u'eBook,', u'complying', u'with', u'the', u'rules'),
 (u'the', u'previous', u'one--the', u'old', u'editions', u'will', u'be'),
 (u'the',
  u'official',
  u'Project',
  u'Gutenberg-tm',
  u'web',
  u'site',
  u'(www.gutenberg.org),'),
 (u'and', u'licensed', u'works', u'that', u'can', u'be', u'freely'),
 (u'eBook', u'for', u'nearly', u'any', u'purpose', u's

In [59]:
douglass_shakespeare_mentions_trimmed.count()

54

In [60]:
douglass_shakespeare_mentions.count()

1226

In [61]:
#King James Version (KJV) of Bible from Project Guttenberg
%time
if not os.path.exists('kjv.txt'):
    ! wget http://www.gutenberg.org/cache/epub/10/pg10.txt -O kjv.txt

kjv_doc = sc.textFile('kjv.txt')
kjv_words = kjv_doc.flatMap(lambda x :x.split())
kjv_words.cache()
kjv_words.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.82 µs
--2016-12-21 19:23:09--  http://www.gutenberg.org/cache/epub/10/pg10.txt
Resolving www.gutenberg.org... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4452070 (4.2M) [text/plain]
Saving to: “kjv.txt”

100%[======================================>] 4,452,070   3.23M/s   in 1.3s    

2016-12-21 19:23:10 (3.23 MB/s) - “kjv.txt” saved [4452070/4452070]



824146

In [62]:
kjv_sevengrams = kjv_doc.map(lambda x:x.split()) \
    .flatMap(lambda x: [((x[i],x[i+1],x[i+2],x[i+3],x[i+4],x[i+5],x[i+6])) for i in range(0,len(x)-6)])
kjv_sevengrams.take(10)

[(u'The', u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'King'),
 (u'Project', u'Gutenberg', u'EBook', u'of', u'The', u'King', u'James'),
 (u'Gutenberg', u'EBook', u'of', u'The', u'King', u'James', u'Bible'),
 (u'This', u'eBook', u'is', u'for', u'the', u'use', u'of'),
 (u'eBook', u'is', u'for', u'the', u'use', u'of', u'anyone'),
 (u'is', u'for', u'the', u'use', u'of', u'anyone', u'anywhere'),
 (u'for', u'the', u'use', u'of', u'anyone', u'anywhere', u'at'),
 (u'the', u'use', u'of', u'anyone', u'anywhere', u'at', u'no'),
 (u'use', u'of', u'anyone', u'anywhere', u'at', u'no', u'cost'),
 (u'of', u'anyone', u'anywhere', u'at', u'no', u'cost', u'and')]

In [63]:
%time douglass_kjv_mentions = kjv_sevengrams.intersection(douglass_sevengrams)
douglass_kjv_mentions.top(10)

CPU times: user 12 ms, sys: 1 ms, total: 13 ms
Wall time: 20.2 ms


[(u'your', u'written', u'explanation.', u'The', u'person', u'or', u'entity'),
 (u'your', u'efforts', u'and', u'donations', u'can', u'help,', u'see'),
 (u'your', u'country', u'in', u'addition', u'to', u'the', u'terms'),
 (u'you,', u'scribes', u'and', u'Pharisees,', u'hypocrites!', u'for', u'ye'),
 (u'you!)', u'can', u'copy', u'and', u'distribute', u'it', u'in'),
 (u'you', u'wish', u'to', u'charge', u'a', u'fee', u'or'),
 (u'you', u'share', u'it', u'without', u'charge', u'with', u'others.'),
 (u'you', u'received', u'the', u'work', u'from.', u'If', u'you'),
 (u'you', u'received', u'the', u'work', u'electronically,', u'the', u'person'),
 (u'you', u'paid', u'the', u'fee', u'as', u'set', u'forth')]

In [64]:
douglass_kjv_mentions.count()

1298

In [65]:
%time douglass_kjv_mentions_trimmed = douglass_kjv_mentions.subtract(kafka_sevengrams, numPartitions=None)

CPU times: user 19 ms, sys: 1 ms, total: 20 ms
Wall time: 26.6 ms


In [66]:
douglass_kjv_mentions_trimmed.count()

86

In [67]:
douglass_kjv_mentions_trimmed.collect()

[(u'with', u'this', u'eBook', u'or', u'online', u'at', u'www.gutenberg.org'),
 (u'make', u'one', u'proselyte,', u'and', u'when', u'he', u'is'),
 (u'required', u'of', u'us', u'mirth,', u'saying,', u'Sing', u'us'),
 (u'to', u'have', u'done,', u'and', u'not', u'to', u'leave'),
 (u'stranger', u'with', u'thee,', u'and', u'a', u'sojourner,', u'as'),
 (u'sea', u'and', u'land', u'to', u'make', u'one', u'proselyte,'),
 (u'be', u'avenged', u'on', u'such', u'a', u'nation', u'as'),
 (u'they', u'do', u'for', u'to', u'be', u'seen', u'of'),
 (u'thee,', u'O', u'Jerusalem,', u'let', u'my', u'right', u'hand'),
 (u'land', u'to', u'make', u'one', u'proselyte,', u'and', u'when'),
 (u'not', u'remember', u'thee,', u'let', u'my', u'tongue', u'cleave'),
 (u'their', u'works', u'they', u'do', u'for', u'to', u'be'),
 (u'us', u'required', u'of', u'us', u'mirth,', u'saying,', u'Sing'),
 (u'the',
  u'official',
  u'Project',
  u'Gutenberg-tm',
  u'web',
  u'site',
  u'(www.gutenberg.org),'),
 (u'first', u'which', u'